In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../src')

from DFutils import DataFrameUtils

In [2]:
df_offer = pd.read_csv('../../data/00_raw/ofertas_relampago.csv')

Siguiendo la analysis de lo notebook 0:
- Converter los tipos de datos de tiempo a datetime
- deletar la columna OFFER_TYPE
- Se añadirá un tipo 'Otros' para la columna ORIGIN

In [3]:
df_offer['OFFER_START_DATE'] = pd.to_datetime(df_offer['OFFER_START_DATE'])
df_offer['OFFER_START_DTTM'] = pd.to_datetime(df_offer['OFFER_START_DTTM'])
df_offer['OFFER_FINISH_DTTM'] = pd.to_datetime(df_offer['OFFER_FINISH_DTTM'])

In [4]:
df_offer.drop(columns = 'OFFER_TYPE',inplace=True)

In [5]:
df_offer.loc[df_offer['ORIGIN'].isnull(),'ORIGIN'] = 'Otros'

En la teoria, las ofertas relampago son ofertas que duran algunas horas, Se va crear variables de tiempo como:
- Horas que lo producto tuve la promo
- Cual dia de la semana
- Cual mes, si existir más de uno mes

In [6]:
df_offer.sample(3)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
3433,2021-06-26,2021-06-26 19:00:00+00:00,2021-06-26 19:00:05+00:00,5,5,NaN,NaN,Otros,free_shipping,AUTOPARTS,ACC,MLM-VEHICLE_LED_BULBS
3567,2021-06-26,2021-06-26 13:00:00+00:00,2021-06-26 19:00:03+00:00,10,10,NaN,NaN,Otros,free_shipping,SPORTS,APP & SPORTS,MLM-LEGGINGS
44135,2021-07-12,2021-07-12 13:00:00+00:00,2021-07-12 19:00:04+00:00,10,3,14.36,7.0,Otros,none,TOYS AND GAMES,T & B,MLM-FIDGET_SPINNERS


In [7]:
df_offer['DURATION_HOURS'] = DataFrameUtils.compute_duration_hours(df_offer,'OFFER_START_DTTM','OFFER_FINISH_DTTM')

In [8]:
df_offer.sample(3)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,DURATION_HOURS
7523,2021-07-03,2021-07-03 13:00:00+00:00,2021-07-03 19:00:01+00:00,15,2,85.95,13.0,Otros,free_shipping,COMPUTERS,CE,MLM-HEADPHONES,6.000278
8573,2021-07-07,2021-07-07 19:00:00+00:00,2021-07-08 01:00:01+00:00,5,5,NaN,NaN,Otros,none,MOBILE,CE,MLM-CELLPHONE_ACCESSORIES,6.000278
23121,2021-06-07,2021-06-07 15:00:00+00:00,2021-06-07 23:00:00+00:00,15,15,NaN,NaN,A,free_shipping,TOOLS AND CONSTRUCTION,HOME & INDUSTRY,MLM-ELECTRIC_PAINT_SPRAYERS,8.000000


In [9]:
df_offer['START_DAY_OF_WEEK'] = df_offer['OFFER_START_DTTM'].dt.day_of_week
df_offer['START_MONTH'] = df_offer['OFFER_START_DTTM'].dt.month

La funccion de pandas dt.day_of_week empeza con lunes=0 y termina con domingo=6

In [10]:
df_offer.sample(3)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,DURATION_HOURS,START_DAY_OF_WEEK,START_MONTH
22674,2021-07-28,2021-07-28 19:00:00+00:00,2021-07-29 01:00:01+00:00,5,4,1.95,1.0,Otros,none,MOTOPARTS,ACC,MLM-MOTORCYCLE_GLOVES,6.000278,2,7
16082,2021-06-23,2021-06-23 12:00:00+00:00,2021-06-23 20:00:01+00:00,5,5,NaN,NaN,A,none,APPAREL ACCESORIES,APP & SPORTS,MLM-HATS_AND_CAPS,8.000278,2,6
3798,2021-06-26,2021-06-26 19:00:00+00:00,2021-06-27 01:00:02+00:00,5,5,NaN,NaN,Otros,none,APPAREL ACCESORIES,APP & SPORTS,MLM-CHARMS_AND_MEDALS,6.000556,5,6


<b> Se va crear columnas de relacionadas a monetary como:
- Cantidad vendidas por hora
- Valor vendido por hora

Nota que aun Se va tener que agregar y despues tener las cuentas tambien dado que nuestro dado esta por linea de ventas y Se necesita entender lo comportamiento de los grupos de las ofertas

In [11]:
df_offer['QUANTITY_PER_HOUR'] = df_offer['SOLD_QUANTITY'] / df_offer['DURATION_HOURS']
df_offer['AMOUNT_PER_HOUR'] = df_offer['SOLD_AMOUNT'] / df_offer['DURATION_HOURS']

In [12]:
df_offer.sample(3)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,DURATION_HOURS,START_DAY_OF_WEEK,START_MONTH,QUANTITY_PER_HOUR,AMOUNT_PER_HOUR
22085,2021-07-28,2021-07-28 19:00:00+00:00,2021-07-29 01:00:03+00:00,15,14,5.75,1.0,Otros,free_shipping,PERSONAL CARE,BEAUTY & HEALTH,MLM-SKIN_CARE_SUPPLIES,6.000833,2,7,0.166644,0.958200
21430,2021-06-27,2021-06-27 07:00:00+00:00,2021-06-27 13:00:03+00:00,5,4,4.54,1.0,Otros,none,APPAREL,APP & SPORTS,MLM-UNDERPANTS,6.000833,6,6,0.166644,0.756562
48441,2021-06-19,2021-06-19 15:00:00+00:00,2021-06-19 23:00:00+00:00,10,10,NaN,NaN,A,free_shipping,TOYS AND GAMES,T & B,MLM-KITCHEN_PLAYSETS,8.000000,5,6,NaN,NaN


Se va salvar y empezar lo EDA

In [13]:
df_offer.to_parquet('../../data/01_processed/ofertas_relampago.parquet',index=False)